In [0]:
# dbutils.fs.mount(
#   source = "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net",
#   mount_point = "/mnt/iotdata",
#   extra_configs = {"fs.azure.account.key.<storage-account-name>.blob.core.windows.net":dbutils.secrets.get(scope = "<scope-name>", key = "<key-name>")})

In [0]:
dbutils.fs.mount(
source = "wasbs://tokyodata@tokyostoarge.blob.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyo_olymic",
extra_configs = {"fs.azure.account.key.tokyostoarge.blob.core.windows.net":"sk7hK1X3dHoLlmLE3WvTPlTYyoVNhBQa3dBQ9PIxGUoFAgliJz9haIJHJ+F/QD+nGk7Tk8rPMW0q+AStSbRTAw=="})
  

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1560701934220911>:1
----> 1 dbutils.fs.mount(
      2 source = "wasbs://tokyodata@tokyostoarge.blob.core.windows.net", # contrainer@storageacc
      3 mount_point = "/mnt/tokyo_olymic",
      4 extra_configs = {"fs.azure.account.key.tokyostoarge.blob.core.windows.net":"sk7hK1X3dHoLlmLE3WvTPlTYyoVNhBQa3dBQ9PIxGUoFAgliJz9haIJHJ+F/QD+nGk7Tk8rPMW0q+AStSbRTAw=="})

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o397.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyo_olymic; nested exception is: 
	java.lang.IllegalArgume

In [0]:
%fs ls "/mnt/tokyo_olymic"

path,name,size,modificationTime
dbfs:/mnt/tokyo_olymic/raw_layer/,raw_layer/,0,0
dbfs:/mnt/tokyo_olymic/transformed_layer/,transformed_layer/,0,0


In [0]:
%fs ls dbfs:/mnt/tokyo_olymic/raw_layer/

path,name,size,modificationTime
dbfs:/mnt/tokyo_olymic/raw_layer/Athletes.csv,Athletes.csv,418492,1708851178000
dbfs:/mnt/tokyo_olymic/raw_layer/Coaches.csv,Coaches.csv,16889,1708851193000
dbfs:/mnt/tokyo_olymic/raw_layer/EntriesGender.csv,EntriesGender.csv,1123,1708851208000
dbfs:/mnt/tokyo_olymic/raw_layer/Medals.csv,Medals.csv,2414,1708851222000
dbfs:/mnt/tokyo_olymic/raw_layer/Teams.csv,Teams.csv,35270,1708851236000


In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo_olymic/raw_layer/Athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo_olymic/raw_layer/Coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo_olymic/raw_layer/EntriesGender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo_olymic/raw_layer/Medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyo_olymic/raw_layer/Teams.csv")

In [0]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [0]:
athletes.printSchema()
coaches.printSchema()


root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
entriesgender.printSchema()
medals.printSchema()
teams.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [0]:
# Find the top countries with the highest number of gold medals
medals.show(10)
top_gold=medals.orderBy("Gold",ascending=False).select("Team_Country","Gold")
top_gold.show(10,truncate=False)

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
+----+--------------------+----+------+------+-----+-------------+
only showing top 10 rows

+--------------------------+----+
|T

In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender.withColumn(
    'Avg_Female', entriesgender['Female'] / entriesgender['Total']
).withColumn(
    'Avg_Male', entriesgender['Male'] / entriesgender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
#teams.show(20)
teams_by_country_df=teams.groupBy(col("TeamName")).count().orderBy("count",ascending=False)
teams_by_country_df.show()

+-----------------+-----+
|         TeamName|count|
+-----------------+-----+
|            Japan|   45|
|    United States|   43|
|            Italy|   34|
|           France|   32|
|          Germany|   32|
|        Australia|   32|
|              ROC|   31|
|            China|   30|
|    Great Britain|   28|
|           Canada|   27|
|      Netherlands|   24|
|           Brazil|   21|
|           Poland|   19|
|            Spain|   18|
|Republic of Korea|   17|
|      New Zealand|   13|
|          Hungary|   13|
|     South Africa|   12|
|            Egypt|   11|
|          Belgium|   10|
+-----------------+-----+
only showing top 20 rows



In [0]:
teams.show(20)
teams_by_country_nd_Event_df=teams.groupBy("TeamName","Event").count().select("TeamName","Event","count").orderBy("count",ascending=False)
teams_by_country_nd_Event_df.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

In [0]:
teams_by_country_nd_Event_df_gender = teams_by_country_nd_Event_df.groupBy("TeamName").pivot("Event").sum("count").select("TeamName","Men","Women").orderBy("Men",ascending=False)

display(teams_by_country_nd_Event_df_gender)

TeamName,Men,Women
Japan,8,8
Argentina,6,2
Spain,5,4
Australia,5,5
Germany,4,1
France,4,4
United States,4,6
South Africa,4,2
Italy,3,2
Canada,3,4


In [0]:
coaches.show(10)
coaches_by_country_discipline=coaches.groupBy("Country","Discipline").count().select("Country","Discipline","count").orderBy(["Country","Discipline"],ascending=False)
coaches_by_country_discipline.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    null|
|           ABE Junya|               Japan|       Volleyball|    null|
|       ABE Katsuhiko|               Japan|       Basketball|    null|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    null|
|          AGEBA Yuya|               Japan|       Volleyball|    null|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
+--------------------+--------------------+-----------------+--------+
only s

In [0]:
athletes.printSchema()
athletes.show(10)
athletes_by_country_cnt=athletes.groupBy("Country").count().orderBy("Count",ascending=False)
athletes_by_country_cnt.show()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)

+-----------------+--------------------+-------------------+
|       PersonName|             Country|         Discipline|
+-----------------+--------------------+-------------------+
|  AALERUD Katrine|              Norway|       Cycling Road|
|      ABAD Nestor|               Spain|Artistic Gymnastics|
|ABAGNALE Giovanni|               Italy|             Rowing|
|   ABALDE Alberto|               Spain|         Basketball|
|    ABALDE Tamara|               Spain|         Basketball|
|        ABALO Luc|              France|           Handball|
|     ABAROA Cesar|               Chile|             Rowing|
|    ABASS Abobakr|               Sudan|           Swimming|
| ABBASALI Hamideh|Islamic Republic ...|             Karate|
|    ABBASOV Islam|          Azerbaijan|          Wrestling|
+-----------------+--------------------+-------------------+
only showing top 

In [0]:
%fs ls "/mnt/tokyoolymic/transformed-data"

path,name,size,modificationTime
dbfs:/mnt/tokyoolymic/transformed-data/athletes_by_country_cnt/,athletes_by_country_cnt/,0,0
dbfs:/mnt/tokyoolymic/transformed-data/average_entries_by_gender/,average_entries_by_gender/,0,0
dbfs:/mnt/tokyoolymic/transformed-data/coaches_by_country_discipline/,coaches_by_country_discipline/,0,0
dbfs:/mnt/tokyoolymic/transformed-data/teams_by_country_df/,teams_by_country_df/,0,0
dbfs:/mnt/tokyoolymic/transformed-data/teams_by_country_nd_Event_df/,teams_by_country_nd_Event_df/,0,0
dbfs:/mnt/tokyoolymic/transformed-data/teams_by_country_nd_Event_df_gender/,teams_by_country_nd_Event_df_gender/,0,0


In [0]:
average_entries_by_gender.write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymic/transformed_layer/average_entries_by_gender")
teams_by_country_df.write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymic/transformed_layer/teams_by_country_df")
teams_by_country_nd_Event_df.write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymic/transformed_layer/teams_by_country_nd_Event_df")
teams_by_country_nd_Event_df_gender.write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymic/transformed_layer/teams_by_country_nd_Event_df_gender")
coaches_by_country_discipline.write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymic/transformed_layer/coaches_by_country_discipline")
athletes_by_country_cnt.write.mode("overwrite").option("header","true").csv("/mnt/tokyo_olymic/transformed_layer/athletes_by_country_cnt")

In [0]:
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/rawcoaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/rawentriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/rawmedals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/rawteams")
     